# 제약주의 임상시험 직후 주가 반응 이벤트 스터디

식약처 의약품 임상시험 리스트(link: https://nedrug.mfds.go.kr/searchClinic) 에서 상장기업을 골라낸 뒤 해당 임상시험 허가/불허 날짜를 찾고 분석에 적절한 종목을 선정한다. 
- 임상시험 허가심사 및 임상시험 결과가 미리 예측 가능하지 않아야 하며
- 임상시험 이외의 외부요인에 주가가 영향을 많이 받지 않는 종목이어야 한다. (가령, 의약품 말고도 화장품/마스크 등 다른 사업을 많이 가지고 있는 경우.) 

해당 날짜의 임상 허가/불허 뉴스가 최초 게시된 시각을 알아낸다. (분 단위)

그 시각 전후의 주가를 잘 관찰하여 적절한 estimation window, event window를 설정한다. 

한 종목에서 임상 뉴스에 대한 주가의 반응이 얼마나 빨리, 얼마나 나타나는지, 종목간의 차이는 어떤지 알아본다. 

## 프로젝트의 해결해야 할 이슈

- 현업에서 일반적으로 바이오/제약 관련 이벤트 스터디가 어떻게 이뤄지고 있는지 확인해야 한다. 
- 아카데믹에서 일반적으로 바이오/제약 관련 이벤트 스터디가 어떻게 이뤄지고 있는지 확인해야 한다. 
- 임상시험 허가/불허가 아닌 임상시험 성공/실패 데이터를 구하는 것이 더 영향력이 크고, 중요할 것으로 생각된다. 
    - 관련하여, 금융위는 앞으로는 투자자의 혼란을 막기 위해 '임상시험 성공'이라는 표현을 사용할 수 없게했다. 그 말은 지금까지의 임상시험 성공/실패 관련 기사나 공시에는 객관적으로 성공/실패를 알기 힘든, 오해를 불러일으킬 수 있는 표현이 사용되었다고 해석할 수 있다. (기사 참고: http://biospectator.com/view/news_view.php?varAtcId=9502) 
- 이벤트 스터디와 관련하여, 어떻게 객관적으로 estimation window, event window를 설정하고 임상시험의 효과를 검증할 수 있을지 고민해봐야 한다. 
- 1분단위 고빈도 주가 데이터라는 점을 고려하여 정상수익률 모형을 어떻게 설정해야 하는지 고민해봐야 한다. 

In [53]:
import win32com.client

import pandas as pd
from sklearn import preprocessing
import re

### 식약처 임상시험 리스트를 불러온다. 

In [54]:
xlsx = pd.ExcelFile('./data_files/nedrug_mfds_ALL_20120101-20200214.xlsx')
mfds_clinical_trials = pd.read_excel(xlsx, 'Sheet0')
mfds_clinical_trials.columns = ['status', 'client', 'product_name', 'ingredients', 'clinical_trial_title', 'stage', 'is_domestic', 'test_institutions', 'approval_date']
'''
진행현황: status,
의뢰자: client,
제품명: product_name,
성분명: ingredients,
임상시험 제목: clinical_trial_title,
임상시험 단계: stage,
개발지역: is_domestic,
실시기관: test_institutions,
승인일: approval_date,
'''
mfds_clinical_trials.head()

,status,client,product_name,ingredients,clinical_trial_title,stage,is_domestic,test_institutions,approval_date
0,승인완료,경희대학교병원,콜리스 주 Colis Inj.(품목기준코드: 201200983),콜리스틴메탄설포네이트나트륨,건강한 자원자를 대상으로 Colistin 약동학 평가 및 신독성 예측 바이오마커를 ...,연구자 임상시험,국내개발,경희대학교병원,2020-02-14
1,승인완료,유유제약,YY두타스테리드정제(가칭) 0.5mg,YY두타스테리드정제(가칭) 0.5mg,㈜유유제약 “YY두타스테리드정제(가칭) 0.5 mg” 과 ㈜글락소스미스클라인 “아보...,1상,국내개발,에이치플러스 양지병원,2020-02-13
2,승인완료,연세대학교의과대학세브란스병원,pemetrexed,pemetrexed,전이성/재발성 연부조직 육종 환자에서 Pemetrexed/Cisplatin 화학요법...,연구자 임상시험,국내개발,"연세대학교의과대학 강남세브란스병원,재단법인아산사회복지재단서울아산병원,삼성서울병원,연...",2020-02-12
3,승인완료,한양대학교병원,UN03,UN03,섬유화를 동반한 비알코올성 지방간염(NASH) 환자에서 UN03의 안전성 평가 및 ...,연구자 임상시험,국내개발,한양대학교병원,2020-02-12
4,승인완료,에스씨엠생명과학주식회사,SCM-AGH,동종골수유래중간엽줄기세포,ADT2002 시험에 참여했던 시험대상자에서 SCM-AGH에 대한 장기 안전성 추적...,연장,국내개발,"인하대학교의과대학부속병원,강동경희대학교의대병원,건국대학교병원,고려대학교의과대학부속안...",2020-02-11


In [55]:
mfds_clinical_trials.describe()

,status,client,product_name,ingredients,clinical_trial_title,stage,is_domestic,test_institutions,approval_date
count,5674,5674,5674,5536,5673,5674,5649,5608,5674
unique,3,586,3352,2943,5613,18,2,2920,1717
top,승인완료,서울대학교병원,MK-3475,MK-3475,건강한 성인 남성에서 음식물이 JLP-1310의 약동학적 특성에 미치는 영향을 평가...,1상,국내개발,서울대학교병원,2019-09-30
freq,2815,192,64,64,3,1631,3298,341,16


<h5 style='color:red'>TODO: Label Encoding</h5>

In [56]:
mfds_clinical_trials.status.unique()

array(['승인완료', '모집완료', '종료'], dtype=object)

In [57]:
mfds_clinical_trials.is_domestic.unique()

array(['국내개발', '국외개발', nan], dtype=object)

In [58]:
mfds_clinical_trials.approval_date = pd.to_datetime(mfds_clinical_trials.approval_date, format='%Y-%m-%d')
mfds_clinical_trials.head()

,status,client,product_name,ingredients,clinical_trial_title,stage,is_domestic,test_institutions,approval_date
0,승인완료,경희대학교병원,콜리스 주 Colis Inj.(품목기준코드: 201200983),콜리스틴메탄설포네이트나트륨,건강한 자원자를 대상으로 Colistin 약동학 평가 및 신독성 예측 바이오마커를 ...,연구자 임상시험,국내개발,경희대학교병원,2020-02-14
1,승인완료,유유제약,YY두타스테리드정제(가칭) 0.5mg,YY두타스테리드정제(가칭) 0.5mg,㈜유유제약 “YY두타스테리드정제(가칭) 0.5 mg” 과 ㈜글락소스미스클라인 “아보...,1상,국내개발,에이치플러스 양지병원,2020-02-13
2,승인완료,연세대학교의과대학세브란스병원,pemetrexed,pemetrexed,전이성/재발성 연부조직 육종 환자에서 Pemetrexed/Cisplatin 화학요법...,연구자 임상시험,국내개발,"연세대학교의과대학 강남세브란스병원,재단법인아산사회복지재단서울아산병원,삼성서울병원,연...",2020-02-12
3,승인완료,한양대학교병원,UN03,UN03,섬유화를 동반한 비알코올성 지방간염(NASH) 환자에서 UN03의 안전성 평가 및 ...,연구자 임상시험,국내개발,한양대학교병원,2020-02-12
4,승인완료,에스씨엠생명과학주식회사,SCM-AGH,동종골수유래중간엽줄기세포,ADT2002 시험에 참여했던 시험대상자에서 SCM-AGH에 대한 장기 안전성 추적...,연장,국내개발,"인하대학교의과대학부속병원,강동경희대학교의대병원,건국대학교병원,고려대학교의과대학부속안...",2020-02-11


In [59]:
mfds_clinical_trials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5674 entries, 0 to 5673
Data columns (total 9 columns):
status                  5674 non-null object
client                  5674 non-null object
product_name            5674 non-null object
ingredients             5536 non-null object
clinical_trial_title    5673 non-null object
stage                   5674 non-null object
is_domestic             5649 non-null object
test_institutions       5608 non-null object
approval_date           5674 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(8)
memory usage: 221.7+ KB


### 대신증권 CYBOS API를 이용해 종목코드 반환

상장되지 않은 client는 자동으로 제외한다. 

In [60]:
## 식약처 client는 xxx(주), xxx주식회사 형식인 경우가 있음. 정식 종목명에선 없어야 한다.

def stock_name_parser(name):
    if '(주)' in name:
        return name.strip('(주)')
    elif '주식회사' in name:
        return name.strip('주식회사')
    else:
        return name

In [61]:
## CYBOS와 연결 확인

instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect) # 1이면 성공

1


In [62]:
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")

In [63]:
def stock_name2ticker(name):
    ticker = instCpStockCode.NameToCode(name)
    if ticker == '':
        return None
    else:
        return ticker

In [64]:
mfds_clinical_trials['ticker'] = mfds_clinical_trials.client.apply(stock_name_parser).apply(stock_name2ticker)
mfds_clinical_trials.ticker.head()

0       None
1    A000220
2       None
3       None
4       None
Name: ticker, dtype: object

In [65]:
mfds_clinical_trials = mfds_clinical_trials.dropna(subset=['ticker'])
mfds_clinical_trials.head()

,status,client,product_name,ingredients,clinical_trial_title,stage,is_domestic,test_institutions,approval_date,ticker
1,승인완료,유유제약,YY두타스테리드정제(가칭) 0.5mg,YY두타스테리드정제(가칭) 0.5mg,㈜유유제약 “YY두타스테리드정제(가칭) 0.5 mg” 과 ㈜글락소스미스클라인 “아보...,1상,국내개발,에이치플러스 양지병원,2020-02-13,A000220
7,승인완료,(주)유한양행,YHP1906,YHP1906,YHP1906정 5밀리그램과 YHR1902정 5밀리그램의 생물학적동등성 평가를 위한...,1상,국내개발,"인산의료재단 메트로병원,바이오코아(주)",2020-02-11,A000100
8,승인완료,동아에스티(주),동아아토르바스타틴정80밀리그램(아토르바스타틴칼슘삼수화물),아토르바스타틴칼슘삼수화물,"동아에스티㈜ ""동아아토르바스타틴정80밀리그램(아토르바스타틴칼슘삼수화물)""과 한국화이...",1상,국내개발,"바이오코아(주),에이치플러스 양지병원",2020-02-11,A170900
12,승인완료,동아에스티(주),슈가논정5밀리그램(에보글립틴타르타르산염),에보글립틴타르타르산염(DA-1229),Metformin과 evogliptin 병용 요법으로 혈당 조절이 불충분한 제2형 ...,3상,국내개발,"한림대학교 성심병원,의료법인명지의료재단명지병원,가톨릭대학교부천성모병원,학교법인가톨릭...",2020-02-07,A170900
15,승인완료,환인제약(주),팔페리돈서방정6밀리그램(팔리페리돈),팔리페리돈,환인제약㈜의 “팔페리돈서방정 6밀리그램(팔리페리돈)”과 ㈜한국얀센의 “인베가서방정 ...,1상,국내개발,"베스티안병원,바이오코아(주)",2020-02-07,A016580
